# Basic pdf functionalities

This notebook contains a set of functionalities related to pdf processing.

In [ ]:
%load_ext autoreload
%autoreload 2

import fitz
from obsidianizer.pdf_tools.annotations import extract_annotation, AnnotationExtractionMode
from obsidianizer.pdf_tools.pages import extract_page_annotations, get_blocks_summary, get_words_data_frame
from obsidianizer.pdf_tools.plots import get_rectangles_from_data_frame
from IPython.display import display

import plotly.graph_objects as go

from obsidianizer.pdf_tools.page_plots import get_page_figure_widget, get_book_figure_widget
from obsidianizer import EXAMPLE_ECCE_HOMMO_PDF_PATH

import fitz
from obsidianizer.obsidian.vault import load_vault, save_vault
from obsidianizer.obsidian.transformations import join_vaults, create_common_words_vault
from obsidianizer.obsidian.journal_tools import get_vault_df_from_journal, get_journal_entries_from_vault
from obsidianizer.latex_tools.utils import load_drafts_entries
from obsidianizer.latex_tools.journal_processing import get_sentences
from obsidianizer.pdf_tools.ecce_homo import is_ecce_hommo_chapter, is_ecce_hommo_subsection
from obsidianizer.pdf_tools.documents import get_book_filtered_blocks, extract_book_annotations
from obsidianizer.obsidian.pdf_tools import get_vault_df_from_pdf, get_vault_df_from_pdf_by_page
from obsidianizer.latex_tools.utils import load_drafts_entries, save_cleaned_sentences_to_latex, print_differences_in_journals
from obsidianizer.obsidian.utils import get_duplicated_vault_keywords, get_vault_df_unique_backlinks, get_backlinks, get_subbacklinks_from_other_backlinks, add_sublinks_to_vault_df

from obsidianizer.nlp.text_cleanup import get_most_used_words
from obsidianizer.nlp.text_cleanup import remove_stop_words_en
import glob
import pandas as pd
import numpy as np
from obsidianizer.utils.unwrapper import unwrap

# 4. Batch processing of pdfs

Ideally we would like to automatically process all the pdfs in a given folder.

In [ ]:
path = "../../../knowledge/Books/Improvement/"

In [ ]:
filepaths = [f for f in glob.glob(path + "**/*.pdf", recursive=True)]

In [ ]:
filepaths

In [ ]:
doc = fitz.open(filepaths[1]) 
book = doc

In [ ]:
chapter_blocks = get_book_filtered_blocks(book, lambda x: "~~~~~~~" in x["words"])

# 5. Class to contain all the information about a given pdf.

In [ ]:
class PDF():
    def __init__(self, filepath: str):
        self.filepath = filepath
        
        

Create this class in the future which will contain all the information necessary to compute the vault, modify it, store the subsections and sections to disk for future reference... Create convention of naming... unique folder with:
- pdf
- annotations
- section
- subsection

Functionalities to read from disk, save and reload, modify pdf... 

# 6 Adding annotations

Sometimes we would like to modify the annotations

# I think I will have to desist  in this idea. Pymupdf cannot do ocr on its own and the tool I use, ocrmypdf, changes the format of the pdf, like the boundaries of the boxes, and the format of the info so it quite hard for me to align them, in the future people should simply ocr the document before they highlight them.

## Maybe there is hope although I should first come up with a way of aligning the test.

## I need to set up all the necesary fields of the annotations and handle subcomments and so on. 

## Copy annotation from one book to the other

In [ ]:
book_with_annotations_no_ocr = fitz.open('../../../knowledge/Books/Improvement/49 laws to power.pdf') 
book_with_ocr_no_annotations = fitz.open('../../../knowledge/Books/Improvement/caca2.pdf') 

In [ ]:
page_index = 19 

annotated_page = book_with_annotations_no_ocr[page_index]
to_be_annotated_page = book_with_ocr_no_annotations[page_index]

annotation = annotated_page.firstAnnot

In [ ]:
annotated_page.getTextBlocks()

In [ ]:
to_be_annotated_page

In [ ]:
annotation.type

In [ ]:
if annotation.type[1] == "Highlight":
    
    # Correct for page missalignment due to ocr modifying the rect of the page.
    diplacement = (to_be_annotated_page.rect - annotated_page.rect)[2:4]
    diplacement = np.array(diplacement)/2
    new_annot_vertices = np.array(annotation.vertices ) + diplacement
    new_annot_vertices = list(new_annot_vertices)
    
    # Create annotation
    new_annot = to_be_annotated_page.addHighlightAnnot(new_annot_vertices)
    
    # Set other things up
    new_annot.set_info(annotation.info)
    new_annot.set_border(annotation.border)
    new_annot.update()

In [ ]:
new_annot_vertices = annotation.vertices
diplacement = (to_be_annotated_page.rect - annotated_page.rect)[2:4]
diplacement = np.array(diplacement)/2

In [ ]:
new_annot_vertices

In [ ]:
to_be_annotated_page.rect

In [ ]:
annotated_page.rect - to_be_annotated_page.rect

In [ ]:
to_be_annotated_page.rect

In [ ]:
new_annot.border


In [ ]:
book_with_ocr_no_annotations.save('../../../knowledge/Books/Improvement/annotated.pdf')

# Idea: Write the section and subsection from highlights with a XX elements in it. 

I do would like to have them separated like that because then you can see more easily what a chapter is about. Having a document which contains references to all the backlinks in the chapter.